In [5]:
import os
import win32com.client
from pywinauto import application
import time
import pandas as pd

# from datetime import time


def connect(reconnect=True):
    # 재연결이라면 기존 연결을 강제로 kill
    if reconnect:
        try:
            os.system('taskkill /IM ncStarter* /F /T')
            os.system('taskkill /IM CpStart* /F /T')
            os.system('taskkill /IM DibServer* /F /T')
            os.system('wmic process where "name like \'%ncStarter%\'" call terminate')
            os.system('wmic process where "name like \'%CpStart%\'" call terminate')
            os.system('wmic process where "name like \'%DibServer%\'" call terminate')
        except:
            pass

    CpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

    if CpCybos.IsConnect:
        print('already connected.')

    else:
        app = application.Application()
        app.start(
            'C:\Daishin\Starter\\ncStarter.exe /prj:cp /id:{id} /pwd:{pwd} /pwdcert:{pwdcert} /autostart'.format(
                id='ssuyan', pwd='tndus1!!', pwdcert='qkrtndus1!!')
        )
        # 연결 될때까지 무한루프
        while True:
            if CpCybos.IsConnect:
                break
            time.sleep(1)

        print('connected.')
    return CpCybos


def _get_ohlcv_by_date(StockChart, code, last_date, start_date, chart_gb, modified='1', field=(0, 1, 2, 3, 4, 5, 8, 9)):
    StockChart.SetInputValue(0, code)
    StockChart.SetInputValue(1, ord('1'))
    StockChart.SetInputValue(2, last_date)
    StockChart.SetInputValue(3, start_date)
    StockChart.SetInputValue(5, field)
    StockChart.SetInputValue(6, ord(chart_gb))
    StockChart.SetInputValue(9, ord(modified))
    StockChart.BlockRequest()
    return _get_df_from_stock_chart(StockChart)

def initIndexInfo(self):
    # 지표 기본 정보, 지표 이름 = 지표 라인 리스트
    self.indexList = {}
    self.indexList['지표선택 없음'] =['없음']
    self.indexList['이동평균(라인1개)'] =['이동평균']
    self.indexList['Stochastic Slow'] = ['SLOW K', 'SLOW D']
    self.indexList['MACD'] = ['MACD', 'SIGNAL', 'OSCILLATOR']
    self.indexList['RSI'] = ['RSI', 'SIGNAL']
    self.indexList['Binary Wave MACD'] = ['BWMACD','SIGNAL', 'OSCILLATOR']
    self.indexList['TSF'] = ['TSF','SIGNAL']
    self.indexList['ZigZag'] = ['ZigZag1', 'ZigZag2']
    self.indexList['Bollinger Band'] = ['Bol상', 'Bol하', 'Bol중']


def _get_df_from_stock_chart(StockChart):
    time.sleep(0.25)
    df = []
    num_col = StockChart.GetHeaderValue(1)
    num_row = StockChart.GetHeaderValue(3)

    for r_idx in range(num_row):
        curr_data = []

        for c_idx in range(num_col):
            curr_data.append(StockChart.GetDataValue(c_idx, r_idx))
        df.append(curr_data)

    df = pd.DataFrame(df, columns=['date', 'time', 'open', 'high', 'low', 'close','volume', 'value'])
    return df


def get_ohlcv_by_date(code, last_date, start_date, chart_gb, modified='1', field=(0, 1, 2, 3, 4, 5, 8 ,9)):
   

    print('get ohlcv by date')
    print(f'code: {code}\nstart_date: {start_date}\nlast_date: {last_date}')
    StockChart = win32com.client.Dispatch("CpSysDib.StockChart")

    df = _get_ohlcv_by_date(StockChart, code, last_date, start_date,
                            chart_gb, modified=modified, field=field)

    last_date = df['date'].iloc[-1]
    last_min = df['time'].iloc[-1]

    df_list = [df]

    while True:
        df = _get_ohlcv_by_date(StockChart, code, int(last_date), start_date,
                            chart_gb, modified=modified, field=field)

        curr_date = df['date'].iloc[-1]
        curr_min = df['time'].iloc[-1]

        if curr_date == last_date and curr_min == last_min:
            break

        last_date = curr_date
        last_min = curr_min

        df_list.append(df)

    df = pd.concat(df_list).drop_duplicates(['date', 'time']).reset_index(drop=True)
    return df


# 이미 연결되어있다면 재연결 x
CpCybos = connect(False)

code = 'A012450'
# 분봉은 최대 2년 데이터 제공
start_date = '20210629'
last_date = '20230628'
chart_gb = 'm'
df = get_ohlcv_by_date(code, last_date, start_date, chart_gb)
print(df)

already connected.
get ohlcv by date
code: A012450
start_date: 20210629
last_date: 20230628
            date  time    open    high     low   close  volume       value
0       20230628  1530  127700  127700  127700  127700   12394  1582710000
1       20230628  1520  127800  127900  127600  127700    2079   265480000
2       20230628  1519  128000  128000  127800  127800    2326   297460000
3       20230628  1518  128200  128300  127700  128000    1057   135220000
4       20230628  1517  128000  128300  127900  128200    6046   774400000
...          ...   ...     ...     ...     ...     ...     ...         ...
188066  20210629   905   46450   46500   46450   46450     882    40970000
188067  20210629   904   46500   46500   46400   46450    3659   169990000
188068  20210629   903   46600   46600   46500   46500    4200   195450000
188069  20210629   902   46600   46650   46550   46600    3590   167290000
188070  20210629   901   46600   46800   46600   46600    3669   171040000

[188071

In [7]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
stock_df = pd.DataFrame({'date':df['date'],'time':df['time'],'open':df['open'],'high':df['high'], 'low':df['low'],'close':df['close'],'value':df['value'], 'volume':df['volume']})
stock_df

stock_df.to_csv('4.csv', index=False)